# Support Vector Machine

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import StandardScaler

from sklearn import svm
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report


from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score
import matplotlib.pyplot as plt
import seaborn as sns

In [13]:
data_url = "http://lib.stat.cmu.edu/datasets/boston"
boston = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
#boston

# EDA

In [14]:
data = np.hstack([boston.values[::2, :], boston.values[1::2, :3]])
#target = boston.values[1::2, 2]
#data
#Convert array into DataFrame and rename columns
boston_data = pd.DataFrame(data, columns =['crime', 'land_zone', 'indus', 'C_River', 'nox','avg_rooms', 'age','dist','r_highway', 'tax', 'pt_ratio','blacks','lstat','price_median_value'])
#boston_data = pd.DataFrame(target, columns =['price_Median_value'])
#boston_data.shape

#Dropping Duplicates
boston_data.duplicated().sum()
#boston_data.drop_duplicates()
#boston_data.shape
#Check Missing Values
boston_data.info()
#Check Null Values
boston_data.isna().sum()
boston_data.isnull().sum()
#boston_data.notna().sum()
#boston_data.notnull().sum()
#boston_data.dropna()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   crime               506 non-null    float64
 1   land_zone           506 non-null    float64
 2   indus               506 non-null    float64
 3   C_River             506 non-null    float64
 4   nox                 506 non-null    float64
 5   avg_rooms           506 non-null    float64
 6   age                 506 non-null    float64
 7   dist                506 non-null    float64
 8   r_highway           506 non-null    float64
 9   tax                 506 non-null    float64
 10  pt_ratio            506 non-null    float64
 11  blacks              506 non-null    float64
 12  lstat               506 non-null    float64
 13  price_median_value  506 non-null    float64
dtypes: float64(14)
memory usage: 55.5 KB


crime                 0
land_zone             0
indus                 0
C_River               0
nox                   0
avg_rooms             0
age                   0
dist                  0
r_highway             0
tax                   0
pt_ratio              0
blacks                0
lstat                 0
price_median_value    0
dtype: int64

# Check Collineariry/Dependability of Variabels

In [27]:
#colinearity check
boston_data.corr()  
#Format the plot background and scatter plots for all the variables
#sns.set_style(style='darkgrid') 
#sns.pairplot(boston_data) #colinearity check

,crime,land_zone,indus,C_River,nox,avg_rooms,age,dist,r_highway,tax,pt_ratio,blacks,lstat,price_median_value
crime,1.000000,-0.200469,0.406583,-0.055892,0.420972,-0.219247,0.352734,-0.379670,0.625505,0.582764,0.289946,-0.385064,0.455621,-0.388305
land_zone,-0.200469,1.000000,-0.533828,-0.042697,-0.516604,0.311991,-0.569537,0.664408,-0.311948,-0.314563,-0.391679,0.175520,-0.412995,0.360445
indus,0.406583,-0.533828,1.000000,0.062938,0.763651,-0.391676,0.644779,-0.708027,0.595129,0.720760,0.383248,-0.356977,0.603800,-0.483725
C_River,-0.055892,-0.042697,0.062938,1.000000,0.091203,0.091251,0.086518,-0.099176,-0.007368,-0.035587,-0.121515,0.048788,-0.053929,0.175260
nox,0.420972,-0.516604,0.763651,0.091203,1.000000,-0.302188,0.731470,-0.769230,0.611441,0.668023,0.188933,-0.380051,0.590879,-0.427321
avg_rooms,-0.219247,0.311991,-0.391676,0.091251,-0.302188,1.000000,-0.240265,0.205246,-0.209847,-0.292048,-0.355501,0.128069,-0.613808,0.695360
age,0.352734,-0.569537,0.644779,0.086518,0.731470,-0.240265,1.000000,-0.747881,0.456022,0.506456,0.261515,-0.273534,0.602339,-0.376955
dist,-0.379670,0.664408,-0.708027,-0.099176,-0.769230,0.205246,-0.747881,1.000000,-0.494588,-0.534432,-0.232471,0.291512,-0.496996,0.249929
r_highway,0.625505,-0.311948,0.595129,-0.007368,0.611441,-0.209847,0.456022,-0.494588,1.000000,0.910228,0.464741,-0.444413,0.488676,-0.381626
tax,0.582764,-0.314563,0.720760,-0.035587,0.668023,-0.292048,0.506456,-0.534432,0.910228,1.000000,0.460853,-0.441808,0.543993,-0.468536


# Remove outliers

In [4]:
#3. remove outlier using median imputation

#plt.figure(figsize=(17,1))
for var in range(0,boston_data.shape[1]):
    temp=boston_data.columns[var]
    lower_quant=boston_data[temp].quantile(0.25)
    upper_quant=boston_data[temp].quantile(0.75)
    med=boston_data[temp].median()
    #print("\nColumn :",temp)
    #print("Lower Quantile:",lower_quant, "Upper Quantile :",upper_quant,"Median :",med)
    
    boston_data[temp] = np.where( (boston_data[temp] <lower_quant), med , boston_data[temp]) 
    #boston_data[temp] = np.where( (boston_data[temp] >upper_quant), med , boston_data[temp])
    #print("\n",boston_data.columns[var])
    #sns.boxplot(data=boston_data[temp])
   # plt.show()
        
#After Outlier Median imputation 
boston_clean=boston_data


# Split dataset

In [15]:
#Split Data into X and y
array = boston_data.values
X = array[:, 0:13]
y = array[:, 13]
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.33)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((339, 13), (339,), (167, 13), (167,))

# Grid Search CV

In [19]:
clf = SVR()
param_grid = [{'kernel':['rbf'],'gamma':[50,5,10,0.5],'C':[15,14,13,12,11,10,0.1,0.001] }]
gsv = GridSearchCV(clf,param_grid,cv=10)
gsv.fit(X_train,y_train)

GridSearchCV(cv=10, estimator=SVR(),
             param_grid=[{'C': [15, 14, 13, 12, 11, 10, 0.1, 0.001],
                          'gamma': [50, 5, 10, 0.5], 'kernel': ['rbf']}])

gsv.best_params_ , gsv.best_score_ 


({'C': 14, 'gamma': 0.5, 'kernel': 'rbf'}, -0.024793208657689126)

# Prepare Model

In [20]:
clf = SVR(C= 14, gamma = 0.5)
clf.fit(X_train , y_train)
y_pred = clf.predict(X_test)

In [24]:
#predicting on the test set and creating submission file
data_pred=pd.DataFrame(data=np.array([[0.25651,0.0,9.69,0.0,0.538,6.5750,65.2,4.09000,5.0,296.0,19.05,391.44,11.36]]),columns=['crime','land_zone','indus','C_River','nox','avg_rooms','age','dist','r_highway','tax','pt_ratio','blacks','lstat'])
#data_pred=pd.DataFrame(data=np.array([[6.25651,5.0,2.07,3.0,6.469,4.4210,78.5,3.96710,3.0,330.1,30.80,391.48,15.14]]),columns=['crime','land_zone','indus','C_River','nox','avg_rooms','age','dist','r_highway','tax','pt_ratio','blacks','lstat'])

clf = SVR(C= 14, gamma = 0.5)
clf.fit(X_train , y_train)

y_pred = clf.predict(X_train)

accuracy=clf.score(X_test, y_test)*100

print("\nAccuracy of Model is :",accuracy)
print("\nPrediction is :",y_pred)




Accuracy of Model is : -0.5322751693929595

Prediction is : [15.69987176 22.10005594 13.9999138  35.67663907 14.50049569 35.67684669
 35.67663914 33.09985526 35.67666293 17.50017833 11.00049758 21.67663907
 22.19990739 11.99997961 25.09984487 10.89994865 21.500454   20.89997066
 30.39986418 17.90045286 35.67664476 19.69958049 20.79998456 22.80049569
 28.89998989 27.00030612 11.89996803 22.69986626 20.09996729 19.60051041
 24.00017668 24.89986414 28.60000015 21.67663907 24.89985519 11.10013532
 18.99991787 19.79987096 28.29991168 35.67663907 13.89986418 22.39960078
 13.19977476 11.79998989 21.69986997 19.69991168 23.6999154  19.40045139
 23.29991168 34.79992505 21.50045286 34.80047844 23.4999191  35.67663907
 14.50044998 22.99996391 16.69987471 26.59997082 23.59987096 21.20049517
 31.10011801 21.80011801 23.29992505 26.49996879 24.69999051 24.69984689
 23.39981823 18.49999113 33.70011801 24.89992736 21.89991658 20.09994865
 20.19997068 22.50017821 14.69994865 10.2999732  35.29986694 20